In [1]:
from vpython import *
import numpy as np
from matplotlib import pyplot as plt
import math
import random

<IPython.core.display.Javascript object>

## Random Walker 

In [5]:
scene = canvas()
#use different seeds
np.random.seed(12345)

n_simulations = 5 
n_steps = 100  

total_distance = 0
total_radius = 0

cyl = cylinder(pos=vector(0,0,0), radius=.5, length=0.5, make_trail=True)

for i in range(n_simulations):
    for j in range(n_steps):
        rate(10)
        rand_direction = int(np.random.uniform(low=0,high=4))
        #0 = right, 1 = left, 2 = up, 3 = down
        if rand_direction == 0:
            cyl.pos.x += 1
        elif rand_direction == 1:
            cyl.pos.x -= 1
        elif rand_direction == 2:
            cyl.pos.y += 1
        else:
            cyl.pos.y -= 1   
        total_distance += mag(cyl.pos)  
    total_radius += mag(cyl.pos)  

average_distance = total_distance / (n_simulations * n_steps)
average_radius = total_radius / n_simulations

expected_radius = np.sqrt(n_steps)

print("Average distance walked:", average_distance)
print("Average endpoint radius:", average_radius)
print("Expected endpoint radius:", expected_radius)

    

<IPython.core.display.Javascript object>

Average distance walked: 7.33468432963379
Average endpoint radius: 7.977854935220721
Expected endpoint radius: 10.0


The average endpoint radius for my random walker is very close to the expected endpoint radius, with the average endpoint radius varying slightly over multiple runs due to its randomness and the expected endpoint radius remaininng constant as N is remained constant.

## References

https://en.wikipedia.org/wiki/Random_walk

## 1D Cellular Automaton Rule 30

In [7]:
scene = canvas()
# Set up the initial row of cells
N = 101
cells = np.zeros(N, dtype=int)
#One live cell in center
cells[N//2] = 1

cell_width = 10
cell_height = 10
cell_spacing = 2
cell_objects = []


previous_cells = cells.copy()
for i in range(N):
    cell = box(pos=vector(i*cell_width, 0, 0), size=vector(cell_width,cell_height, 0), color=color.white)
    cell_objects.append(cell)
scene.camera.follow(cell_objects[0])
for frame in range(2000):
    rate(10)
    new_cells = np.zeros(N, dtype=int)
    for i in range(1, N-1):
        left = cells[i-1]
        middle = cells[i]
        right = cells[i+1]
        #logic expression for rule 30
        new_cells[i] = left ^ (middle | right)
    cells = new_cells
    for i in range(N):
        if cells[i] == 0:
            cell_objects[i].color = color.green
        else:
            cell_objects[i].color = color.red
    if np.array_equal(cells, previous_cells):
        print("STOPPING CRITERION REACHED!!!!")
        break
    previous_cells = cells.copy()

<IPython.core.display.Javascript object>

STOPPING CRITERION REACHED!!!!


I chose rule 30 because there was an easy logic expression I could derive from the rule using a truth table. Rule 30 also results in some unique and interesting patterns and is a very common rule used for cellular automaton. The stopping criterion I used for rule 30 is if the simulation is no longer updating any cells then end the simulation. I did this by creating copies of the current pattern and checking it against the updated pattern and if they are identical I would stop the simulation. 

## References

https://mathworld.wolfram.com/ElementaryCellularAutomaton.html

## Conway's Game of Life

In [2]:
def initial_grid(board):
    # Set still-life block
    board[N//2+2][N//4] = 1
    board[N//2+2][N//4+1] = 1
    board[N//2+1][N//4] = 1
    board[N//2+1][N//4+1] = 1
    
    # Set blinker oscillator
    board[N//2-1][N//2-2] = 1
    board[N//2-1][N//2-1] = 1
    board[N//2-1][N//2] = 1
    
    # Set glider spaceship
    board[N//4][3*N//4] = 1
    board[N//4][3*N//4+1] = 1
    board[N//4+1][3*N//4-1] = 1
    board[N//4+1][3*N//4] = 1
    board[N//4+2][3*N//4+1] = 1
    return board

In [4]:
scene = canvas()
N = 30
cell_width = 10
cell_height = 10
cells = np.zeros((N,N),dtype=int)
cell_objects = []
for i in range(N):
    row= []
    for j in range(N):
        square = box(pos=vector(i*cell_width, j*cell_height, 0), size=vector(cell_width, cell_height, 0), color=color.white)
        row.append(square)
    cell_objects.append(row)
cells = initial_grid(cells)
t = 0
deltat = .1
while t < 10:
    rate(10)
    new_cells = np.zeros((N, N), dtype=int)
    for i in range(N):
        for j in range(N):
            #count live neighbors
            live_neighbors = 0
            for dir_i in [-1, 0, 1]:
                for dir_j in [-1, 0, 1]:
                    #skip current cell
                    if dir_i == 0 and dir_j == 0:
                        continue
                    neighbors_i = i + dir_i
                    neighbors_j = j + dir_j
                    #skip neighbors off grid
                    if neighbors_i < 0 or neighbors_i >= N or neighbors_j < 0 or neighbors_j >= N:
                        continue
                    #count the live neighbors
                    if cells[neighbors_i][neighbors_j] == 1:
                        live_neighbors += 1
            #Conway's game of life rules for updating cells
            if cells[i][j] == 1:
                if live_neighbors < 2 or live_neighbors > 3:
                    new_cells[i][j] = 0
                    cell_objects[i][j].color = color.green
                else:
                    new_cells[i][j] = 1
                    cell_objects[i][j].color = color.red
            else:
                if live_neighbors == 3:
                    new_cells[i][j] = 1
                    cell_objects[i][j].color = color.red
                else:
                    new_cells[i][j] = 0
                    cell_objects[i][j].color = color.green
    cells = new_cells
    t = t + deltat

    


<IPython.core.display.Javascript object>

## Validation Process

The validation process I used was to run through different patterns like the still-life patterns which I knew should not move as the simulation is running, the oscillator patterns which I knew would update between two different positions, and the spaceship patterns which I knew would move across the screen and compare their expected behavior to the behavior observed when playing my simulation. 

## References

https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life

https://playgameoflife.com/

## Asynchronous Cellular Automaton - Random Independent Scheme

In [9]:
scene = canvas()
N = 30
cell_width = 10
cell_height = 10
cells = np.zeros((N,N),dtype=int)
cell_objects = []
for i in range(N):
    row= []
    for j in range(N):
        square = box(pos=vector(i*cell_width, j*cell_height, 0), size=vector(cell_width, cell_height, 0), color=color.white)
        row.append(square)
    cell_objects.append(row)
cells = initial_grid(cells)
for i in range(N):
    for j in range(N):
        if cells[i][j] == 1:
            cell_objects[i][j].color = color.red
        else:
            cell_objects[i][j].color = color.green
t = 0
deltat = .1
while t < 500:
    rate(100)
    new_cells = np.zeros((N, N), dtype=int)
    rand_row = random.randint(0,N-1)
    rand_col = random.randint(0,N-1)
    live_neighbors = 0
    for dir_i in [-1, 0, 1]:
        for dir_j in [-1, 0, 1]:
            #skip current cell
            if dir_i == 0 and dir_j == 0:
                continue
            neighbors_i = rand_row + dir_i
            neighbors_j = rand_col + dir_j
            #skip neighbors off grid
            if neighbors_i < 0 or neighbors_i >= N or neighbors_j < 0 or neighbors_j >= N:
                continue
            #count the live neighbors
            if cells[neighbors_i][neighbors_j] == 1:
                live_neighbors += 1
#     print("live_neighbors: " + str(live_neighbors))
    #Conway's game of life rules for updating cells
    if cells[rand_row][rand_col] == 1:
        if live_neighbors < 2 or live_neighbors > 3:
            cells[rand_row][rand_col] = 0
            cell_objects[rand_row][rand_col].color = color.green
        else:
            cells[rand_row][rand_col] = 1
            cell_objects[rand_row][rand_col].color = color.red
    else:
        if live_neighbors == 3:
            cells[rand_row][rand_col] = 1
            cell_objects[rand_row][rand_col].color = color.red
        else:
            cells[rand_row][rand_col] = 0
            cell_objects[rand_row][rand_col].color = color.green
    t = t + deltat
#     cells = new_cells
print("Simulation stopped!!!")

<IPython.core.display.Javascript object>

Simulation stopped!!!


## Results

I chose random independent scheme because it was fairly easy to implement and easy to understand what is happening with these asynchrnous updates. I also only chose to update one cell at random at a time so it is easier to follow and interpret the changes made as the simulation goes on. Asynchronous results in different patterns when ran over multiple iterations because we are now selecting what random cell we are updating according to the rules instead of looking at every cell in the grid for updating with the synchronouse model. The still life, however, remained the same for both synchronous and asynchronous.